<a href="https://colab.research.google.com/github/aakarshhh/AI_ML/blob/main/Model_Update_StockLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
pip install yfinance 

In [139]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as dr
import datetime as dt 
import yfinance as yf
import tensorflow as tf
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import  Dense , Dropout ,LSTM ,BatchNormalization

In [140]:
pred_days = 8
d= dt.timedelta(days = 15)
start = dt.datetime.now()- d 
end = dt.datetime.now()
start, end

(datetime.datetime(2021, 4, 16, 18, 50, 52, 434476),
 datetime.datetime(2021, 5, 1, 18, 50, 52, 434518))

In [141]:
company=''
for i in pd.read_excel('/content/drive/MyDrive/SentStock/tickers.xlsx',header=None)[0] :
  company+=str(i).strip()+' '
company  
data = yf.download(
        tickers = f'{company}',
        start =start, end=end,
        group_by = 'ticker',
        auto_adjust = True,
        prepost = True)
data = pd.DataFrame(data)
data = data[-(pred_days+1):]
param = pd.read_csv('/content/drive/MyDrive/SentStock/param.csv',header=0).rename(index={0:'mean',1:'std',2:'pred', 3:'real'})

[*********************100%***********************]  80 of 80 completed


In [142]:
from sklearn.preprocessing import StandardScaler

In [143]:
model= Sequential()
model.add(Dense(16, activation='relu',input_shape= ( pred_days , 1)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(LSTM( units = 16 , return_sequences=True, recurrent_dropout=0.2))
model.add(LSTM(units = 64 ,return_sequences=True , recurrent_dropout=0.2))
model.add(LSTM(units = 32  , recurrent_dropout=0.15))
model.add(Dense(8, activation='relu'))
model.add(Dense(units = 1))
model.compile(loss = 'mse' , optimizer = 'adam' )

In [144]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 8, 16)             32        
_________________________________________________________________
dense_16 (Dense)             (None, 8, 32)             544       
_________________________________________________________________
dense_17 (Dense)             (None, 8, 16)             528       
_________________________________________________________________
lstm_7 (LSTM)                (None, 8, 16)             2112      
_________________________________________________________________
lstm_8 (LSTM)                (None, 8, 64)             20736     
_________________________________________________________________
lstm_9 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_18 (Dense)             (None, 8)                

In [145]:
model.load_weights('/content/drive/MyDrive/SentStock/stock_lstm.keras')

In [146]:
sc = StandardScaler()
def train_model(x,y,ep = 7):
  model.fit(x,y, epochs=ep , batch_size = 1 , verbose=2 )

def save_plot(y,p,name = "default"):
  plt.figure(dpi=100)
  plt.plot(y,color="black",label = "Actual Data")
  plt.plot(p, color="green",label = "Predicted Data")
  plt.legend() 
  plt.savefig(fname = f'/content/drive/MyDrive/SentStock/Pred/{name}_'+end.strftime("%m_%d_%Y")+'.jpeg')
  plt.show()

def norm(x , c ):
  sc.mean_= param[f'{c}']['mean']
  sc.scale_= param[f'{c}']['std']
  return sc.transform(x)

def scaled(y , c ):
  sc.mean_= param[f'{c}']['mean']
  sc.scale_= param[f'{c}']['std']
  return sc.inverse_transform(y)

In [147]:
param

,T,TMUS,USM,VZ,AAPL,MSFT,AMZN,GOOGL,FB,V,NVDA,MA,PYPL,NFLX,BRK-B,JPM,BAC,WFC,MS,C,BLK,AMT,PLD,CCI,EQIX,SPG,PSA,WELL,EQR,AVB,SBAC,XOM,CVX,NEP,SO,D,DUK,COP,AEP,KMI,EXC,CVS,UNH,MCK,ABC,CI,CAH,ANTM,JNJ,CNC,HUM,THO,ALGN,PPC,PATK,LCII,SHW,SAFM,LEA,CLX,LANC,DIS,CMCSA,NKE,CHTR,SNE,ATVI,MAR,LVS,VIAC,BTI,MO,DEO,EL,BUD,PM,PEP,KO,PG,GME
mean,24.450321,44.566161,38.059406,37.570379,32.072065,64.590405,859.369213,727.662941,124.157318,80.999486,99.722039,116.584120,83.916711,145.883873,141.482095,62.444389,17.184559,36.487451,30.929734,46.085706,303.631135,105.676356,44.389662,75.573585,283.263261,114.495597,151.183597,49.127854,48.782109,130.931329,120.746395,60.911654,83.809010,36.214221,36.889450,53.265901,59.284802,46.517598,49.959130,19.747635,30.968433,57.205391,130.421346,130.020048,65.442161,115.982080,48.802377,142.120139,90.621119,30.825420,173.509302,56.173065,119.463449,16.363864,27.223442,57.200768,89.805932,82.121464,92.642248,100.265713,94.423348,81.597940,25.472662,49.675375,221.007276,33.602675,33.912597,71.148429,43.917922,39.736406,36.238323,33.956224,103.028962,93.439237,77.917745,63.488579,83.154240,34.995058,72.478901,18.275569
std,5.514537,28.045047,5.213497,11.366281,25.007621,52.627606,816.542032,397.142725,68.108903,57.124041,126.155829,91.676310,47.725608,146.987413,49.514474,30.069237,7.548420,11.135655,12.005330,12.739177,137.288725,61.192043,22.803271,37.074757,187.030403,35.195980,51.230161,14.343507,14.005995,37.052750,75.219011,10.374810,16.986901,11.747838,10.383461,16.064420,17.322228,10.976952,20.818898,5.423961,6.757503,19.644866,90.834072,44.225281,24.807880,59.125094,14.983675,85.072810,33.392605,21.011788,107.087471,29.470035,115.159236,8.736116,21.411300,33.388449,56.590824,36.530747,44.821734,44.993682,41.484377,35.368979,11.696775,28.176860,160.148552,18.379579,23.145703,37.693974,13.661739,14.788797,8.657033,14.054915,31.082462,54.364599,24.004296,16.178060,28.347048,8.890072,24.057638,8.613866
pred,31.270288,133.072205,34.813503,56.941216,132.749603,254.896011,3438.687988,2342.884033,324.362030,232.976166,608.884460,388.098938,266.042664,503.791077,274.940948,153.032227,40.272411,45.200096,82.316475,72.315369,820.559143,252.131897,116.164032,187.224655,713.724915,121.878586,278.646179,75.654480,73.897598,192.054214,297.644806,57.302090,104.128433,74.465652,64.813225,78.748001,99.310799,51.778389,87.542610,16.935249,44.736870,76.160423,399.250671,189.723938,120.942375,249.481186,60.378250,381.841431,163.483185,61.462349,445.939178,140.183228,604.406067,24.283691,90.687927,150.144180,272.468048,162.502625,187.943115,186.619797,185.021317,184.605316,55.956684,131.921341,668.778625,104.563156,91.936409,149.704178,60.846188,40.661884,37.745823,47.029694,181.560486,315.560883,71.297668,94.854477,143.401337,54.081627,132.382599,173.338104
real,31.410000,132.130005,34.130001,57.790001,131.460007,252.179993,3467.419922,2353.500000,325.079987,233.559998,600.380005,382.059998,262.290009,513.469971,274.950012,153.809998,40.529999,45.049999,82.550003,71.239998,819.299988,254.770004,116.529999,189.059998,720.760010,121.739998,281.160004,75.029999,74.230003,192.000000,299.720001,57.240002,103.070000,74.550003,66.169998,79.900002,100.690002,51.139999,88.709999,17.049999,44.939999,76.400002,398.799988,187.559998,120.800003,249.009995,60.340000,379.390015,162.729996,61.740002,445.239990,141.589996,595.530029,23.959999,89.599998,146.500000,273.869995,164.529999,183.839996,182.500000,184.710007,186.020004,56.150002,132.619995,673.450012,100.160004,91.190002,148.520004,61.259998,41.020000,37.509998,47.750000,179.350006,313.799988,70.959999,95.000000,144.160004,53.980000,133.419998,173.589996


In [151]:
cou= 1 
for c in pd.read_excel('/content/drive/MyDrive/SentStock/tickers.xlsx',header=None)[0] :
  print("Training started for "+c)
  c = c.strip()
  sdata =data[f'{c}']['Close'].dropna().values.reshape(-1,1) 
  
  x_train=[]
  y_train=[]
  
  param[f'{c}']['mean'] = sdata.mean()
  param[f'{c}']['std'] = sdata.std() 
  sdata = norm(sdata ,c )
  
  x_train.append(sdata[:-1 ] )
  y_train.append(sdata[-1:])

  x_train,y_train = np.array(x_train) , np.array(y_train)
  x_train= np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

  print((x_train.shape,y_train.shape,c))
  
  pr = scaled(model.predict(x_train),c)
  param[f'{c}']['pred'] = pr
  param[f'{c}']['real'] = data[f'{c}']['Close'][-1:]

  train_model(x_train,y_train)
  print(param[f'{c}']['pred'] , param[f'{c}']['real'])
  print("Training ended for "+c)
  print(cou)
  cou+=1
  print()

Training started for T
((1, 8, 1), (1, 1, 1), 'T')
Epoch 1/7
1/1 - 5s - loss: 0.0104
Epoch 2/7
1/1 - 0s - loss: 2.1851e-08
Epoch 3/7
1/1 - 0s - loss: 4.4601e-04
Epoch 4/7
1/1 - 0s - loss: 2.9638e-04
Epoch 5/7
1/1 - 0s - loss: 0.0026
Epoch 6/7
1/1 - 0s - loss: 0.0058
Epoch 7/7
1/1 - 0s - loss: 7.9719e-04
31.448545455932617 31.40999984741211
Training ended for T
1

Training started for TMUS
((1, 8, 1), (1, 1, 1), 'TMUS')
Epoch 1/7
1/1 - 0s - loss: 2.2807
Epoch 2/7
1/1 - 0s - loss: 2.4859
Epoch 3/7
1/1 - 0s - loss: 1.9548
Epoch 4/7
1/1 - 0s - loss: 1.9939
Epoch 5/7
1/1 - 0s - loss: 1.9690
Epoch 6/7
1/1 - 0s - loss: 1.7486
Epoch 7/7
1/1 - 0s - loss: 1.6855
133.53359985351562 132.1300048828125
Training ended for TMUS
2

Training started for USM
((1, 8, 1), (1, 1, 1), 'USM')
Epoch 1/7
1/1 - 0s - loss: 1.8663
Epoch 2/7
1/1 - 0s - loss: 1.8661
Epoch 3/7
1/1 - 0s - loss: 1.7824
Epoch 4/7
1/1 - 0s - loss: 1.7304
Epoch 5/7
1/1 - 0s - loss: 1.6054
Epoch 6/7
1/1 - 0s - loss: 1.6249
Epoch 7/7
1/1 - 

In [152]:
param = param.dropna(axis=1,)
a = param.loc[['real','pred']]
a.to_excel('/content/drive/MyDrive/SentStock/preds.xlsx')
param.to_csv('/content/drive/MyDrive/SentStock/param.csv',index=False)

In [154]:
model.save('/content/drive/MyDrive/SentStock/stock_lstm.keras')